In [58]:
# Instalamos torchtext que nos facilitará la vida en el pre-procesamiento del formato ConLL.
# !pip install -U torchtext==0.10.0

# Librerias

In [59]:
import random
import os, shutil
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gzip
import os
import shutil
import requests

from operator import attrgetter
from torchtext import vocab, datasets ,data
#from torchtext.legacy import data #, datasets
from seqeval.metrics import f1_score, precision_score, recall_score

In [60]:
# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [61]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))

print(fields)

(('text', <torchtext.data.field.Field object at 0x7fb09054b400>), ('nertags', <torchtext.data.field.Field object at 0x7fb09054bb50>))


####  **SequenceTaggingDataset**

In [62]:
train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="corpus_admins_train.txt",
    validation="corpus_admins_val.txt",
    test="corpus_admins_test.txt",
    fields=fields,
    encoding="utf-8",
    separator="-X- _ " 
    # separator="-X-"
)

In [63]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test: {len(test_data)}")

Numero de ejemplos de entrenamiento: 750
Número de ejemplos de validación: 123
Número de ejemplos de test: 127


Visualizemos un ejemplo

In [64]:
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('15 ', 'B-CANT'), ('MG ', 'B-UND'), ('ORAL ', 'B-VIA_ADMIN')]

#### **Construir los vocabularios para el texto y las etiquetas**

Los vocabularios son los objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [65]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)
# NER_TAGS.build_vocab(train_data_ft)

In [66]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 193
Tokens únicos en NER_TAGS: 8


In [67]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'B-UND',
 'B-CANT',
 'B-VIA_ADMIN',
 'I-UND',
 'O',
 'I-VIA_ADMIN',
 'I-CANT']

In [68]:
# Tokens mas frecuentes (Será necesario usar stopwords, eliminar símbolos o nos entregan información (?) )
TEXT.vocab.freqs.most_common(10)

[('ORAL ', 370),
 ('1 ', 240),
 ('MG ', 212),
 ('INTRAVENOSA ', 193),
 ('COMPRIMIDO ', 175),
 ('UNIDAD ', 137),
 ('2 ', 96),
 ('ML ', 67),
 ('FRASCO ', 62),
 ('INTRAMUSCULAR ', 45)]

In [69]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']
# O_TAG_IDX2 = NER_TAGS.vocab.stoi[' O']

#### **Frecuencia de los Tags**

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [70]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

B-UND	752	31.4%
B-CANT	745	31.1%
B-VIA_ADMIN	744	31.1%
I-UND	68	 2.8%
O	54	 2.3%
I-VIA_ADMIN	29	 1.2%
I-CANT	2	 0.1%


#### DataLoaders

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [71]:
BATCH_SIZE = 32 #16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


#### **Métricas de evaluación**

In [72]:
# Definimos las métricas
def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    # print('pad_idx: ', pad_idx)
    # print(len(preds), len(y_true))
    mask = [(y_true != pad_idx)] 
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).cpu().numpy()
    y_true = y_true.cpu().numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict', zero_division=0)
    precision = precision_score(y_true, y_pred, mode='strict', zero_division=0)
    recall = recall_score(y_true, y_pred, mode='strict', zero_division=0)

    return precision, recall, f1

## Modelos

### **Modelo Baseline**

In [73]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx,
                                      )

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)
        self._init_weights

    def _init_weights(self, m):
        # Inicializamos los pesos como aleatorios
        for name, param in m.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1) 
        
        # Seteamos como 0 los embeddings de UNK y PAD.
        self.embedding.weight.data[UNK_IDX] = torch.zeros(self.embedding_dim)
        self.embedding.weight.data[PAD_IDX] = torch.zeros(self.embedding_dim)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]
        # Pasar los embeddings por la rnn (LSTM)
        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]
        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

### Modelo Gru

In [78]:
# Para crear la red debemos heredar desde nn.Module
class GruNet(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
      super().__init__()

      # Capa de embedding
      self.embedding = nn.Embedding(input_dim,
                                    embedding_dim,
                                    padding_idx=pad_idx,
                                    )
      
      # Capa GRU
      self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout = dropout if n_layers > 1 else 0, bidirectional=bidirectional)
      # Capa de salida
      self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
      #self.relu = nn.ReLU()
      # Dropout
      self.dropout = nn.Dropout(dropout)




    # Definimos las operaciones de las capas sobre el input en el forward.
    def forward(self, text): 
      embedded = self.embedding(text)
      outputs, hidden = self.gru(embedded)
      predictions = self.fc(self.dropout(outputs))
      return predictions


## Funciones Entrenamiento

### **Definimos el entrenamiento de la red**


In [79]:
def train(model, iterator, optimizer, criterion, otag=O_TAG_IDX, ft=False):
    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()
    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # print('1',tags)

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()
        #text = [sent len, batch size]
        # Predecimos los tags del texto del batch.
        predictions = model(text)
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        # print(len(predictions))
        # print(len(tags))
        # print('2',tags)

        #predictions = [sent len * batch size, output dim]
        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags, o_idx=otag)
        # Calculamos los gradientes
        loss.backward()
        # Actualizamos los parámetros de la red
        optimizer.step()

        if ft:
            optimizer.zero_grad()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

### **Definimos la función de evaluación**

In [80]:
def evaluate(model, iterator, criterion, otag=O_TAG_IDX):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags, o_idx=otag)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [81]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [82]:
def train_model(model, model_name, train_iterator, valid_iterator, criterion, optimizer, n_epochs, otag=O_TAG_IDX, ft=False):
  global device

  model = model.to(device)
  criterion = criterion.to(device)
  
  #Agregar early stop
  best_valid_loss = float('inf')
  best_train_loss = float('inf')

  arrayTrainLoss = []
  arrayValidLoss = []

  for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion, otag, ft)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion, otag)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #Nuevo **
    arrayTrainLoss.append(train_loss)
    arrayValidLoss.append(valid_loss)

  #Acá nos aseguramos de que entrene al menos 6 épocas antes de detener el entrenamiento.
    if epoch < 7:
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
      else :
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )

  #Early stop, ve que no vaya en aumento, según el avg de las últimas 4 épocas
    else:
      if train_loss < np.mean(arrayTrainLoss[-4:]) and valid_loss > np.mean(arrayValidLoss[-4:]):
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
        print('Early Stop')
        break

      else:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
  return model, arrayTrainLoss, arrayValidLoss

In [83]:
# def init_weights(m):
#     # Inicializamos los pesos como aleatorios
#     for name, param in m.named_parameters():
#         nn.init.normal_(param.data, mean=0, std=0.1) 
        
#     # Seteamos como 0 los embeddings de UNK y PAD.
#     m.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#     m.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Entrenamiento

### Baseline

In [84]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases
N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = False

model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)
model_name = 'Baseline_Admins'

n_epochs = 10
n_epochs_ft = 50

TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
optimizer = optim.Adam(model.parameters())
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

#model.apply(init_weights(model))
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')
model, arrayTrainLoss, arrayValidLoss = train_model(model, model_name, train_iterator, valid_iterator, criterion, optimizer, n_epochs)

El modelo actual tiene 1,684,020 parámetros entrenables.
Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.655 | Train f1: 0.36 | Train precision: 0.36 | Train recall: 0.36
	 Val. Loss: 1.140 |  Val. f1: 0.59 |  Val. precision: 0.58 | Val. recall: 0.61
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.610 | Train f1: 0.82 | Train precision: 0.81 | Train recall: 0.84
	 Val. Loss: 0.356 |  Val. f1: 0.90 |  Val. precision: 0.88 | Val. recall: 0.91
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.256 | Train f1: 0.93 | Train precision: 0.92 | Train recall: 0.95
	 Val. Loss: 0.247 |  Val. f1: 0.90 |  Val. precision: 0.89 | Val. recall: 0.92
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.183 | Train f1: 0.95 | Train precision: 0.93 | Train recall: 0.96
	 Val. Loss: 0.202 |  Val. f1: 0.95 |  Val. precision: 0.94 | Val. recall: 0.96
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.154 | Train f1: 0.96 | Train precision: 0.95 | Train recall: 0.97
	 Val. Loss: 0.199 |  Val. f1: 0.95 |  Val. precision: 0.95 | Val. r

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.125 | Train f1: 0.96 | Train precision: 0.95 | Train recall: 0.97
	 Val. Loss: 0.180 |  Val. f1: 0.95 |  Val. precision: 0.95 | Val. recall: 0.95
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.121 | Train f1: 0.97 | Train precision: 0.96 | Train recall: 0.98
	 Val. Loss: 0.192 |  Val. f1: 0.95 |  Val. precision: 0.95 | Val. recall: 0.95
Early Stop


In [85]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [86]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
test_loss, test_precision, test_recall, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

Val. Loss: 0.180 |  Val. f1: 0.95 | Val. precision: 0.95 | Val. recall: 0.95
test_loss: 0.351 |  test_f1: 0.93 | test_precision: 0.93 | test_recall: 0.94


### Modelo Gru

In [94]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.3
BIDIRECTIONAL = False

# Creamos nuestro modelo.
model3 = GruNet(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name3 = 'Gru_Admins'  # nombre que tendrá el modelo guardado...
n_epochs3 = 10
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

criterion3 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
optimizer3 = optim.Adam(model3.parameters())
model3, arrayTrainLoss3, arrayValidLoss3 = train_model(model3, model_name3,train_iterator, valid_iterator, criterion3, optimizer3, n_epochs3)

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.556 | Train f1: 0.90 | Train precision: 0.88 | Train recall: 0.91
	 Val. Loss: 0.318 |  Val. f1: 0.89 |  Val. precision: 0.88 | Val. recall: 0.91
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.230 | Train f1: 0.94 | Train precision: 0.93 | Train recall: 0.96
	 Val. Loss: 0.236 |  Val. f1: 0.93 |  Val. precision: 0.92 | Val. recall: 0.93
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.173 | Train f1: 0.96 | Train precision: 0.95 | Train recall: 0.97
	 Val. Loss: 0.205 |  Val. f1: 0.93 |  Val. precision: 0.92 | Val. recall: 0.93
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.163 | Train f1: 0.96 | Train precision: 0.95 | Train recall: 0.96
	 Val. Loss: 0.190 |  Val. f1: 0.95 |  Val. precision: 0.95 | Val. recall: 0.95
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.127 | Train f1: 0.97 | Train precision: 0.97 | Train recall: 0.98
	 Val. Loss: 0.177 |  Val. f1: 0.95 |  Val. precision: 0.96 | Val. recall: 0.95
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.

In [95]:
model3.load_state_dict(torch.load('{}.pt'.format(model_name3)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [96]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model3, valid_iterator, criterion)
test_loss, test_precision, test_recall, test_f1 = evaluate(model3, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

Val. Loss: 0.164 |  Val. f1: 0.96 | Val. precision: 0.96 | Val. recall: 0.95
test_loss: 0.388 |  test_f1: 0.93 | test_precision: 0.93 | test_recall: 0.94


## **Predecir datos**

In [ ]:
def predict_labels(model, iterator, criterion, fields=fields):
    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in iterator:
            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,predictions_batch):
                for word_idx, word_predictions in zip(sentence,sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]
                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]
                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])
    return predictions